In [1]:
import pandas as pd
import numpy as np


In [2]:
#uvoz podataka
#sezona 23/24
df_23_4 = pd.read_csv("la-liga-2023.csv")
#sezona 24/25
df_24_5 = pd.read_csv("la-liga-2024.csv")
#sezona 25/6
df_25_6 = pd.read_csv("la-liga-2025.csv")

In [3]:
#za sezonu 25 6
df_25_6["Home Team"] = df_25_6["Home Team"].replace("Celta", 'RC Celta')
df_25_6["Away Team"] = df_25_6["Away Team"].replace("Celta", 'RC Celta')

## Obavezno!!!

In [4]:
#odrediti duzinu sezone
df_25_6 = df_25_6.head(130)
df_25_6.tail()

,Match Number,Round Number,Date,Location,Home Team,Away Team,Result
125,126,13,23/11/2025 13:00,Estadio Carlos Tartiere,Real Oviedo,Rayo Vallecano,0 - 0
126,122,13,23/11/2025 15:15,Estadio Olímpico de la Cartuja,Real Betis,Girona FC,1 - 1
127,128,13,23/11/2025 17:30,Coliseum,Getafe CF,Atlético de Madrid,0 - 1
128,123,13,23/11/2025 20:00,Estadio Martínez Valero,Elche CF,Real Madrid,2 - 2
129,124,13,24/11/2025 20:00,RCDE Stadium,RCD Espanyol de Barcelona,Sevilla FC,2 - 1


## Priprema Podataka i df-a

In [5]:
#sjedinjavanje ta tri df niza podataka - po osi jedan - odnosno uzduž
df_BL = pd.concat([df_23_4, df_24_5, df_25_6], axis=0)
df_BL = df_BL.drop(["Match Number","Round Number","Date","Location"],axis=1)
df_BL.rename(columns={"Home Team":"H","Away Team":"A"},inplace=True)

rez_dom = df_BL["Result"].str.slice(0,1)
rez_gost = df_BL["Result"].str.slice(4,5)

df_rezultatski = pd.concat([df_BL,rez_dom,rez_gost],axis=1)
df_rezultatski.columns = ["H","A","Result","Rez1","Rez2"]

df1 = df_rezultatski

## Definisanje Funckije za Izracun Forme

In [6]:
def klub(tim):
    # Filtriramo DataFrame
    df = df1[(df1["H"] == tim) | (df1["A"] == tim)].copy()
    
    uslovi = [
    (df['H'] == tim) & (df['Rez1'] < df["Rez2"]),
    (df['A'] == tim) & (df['Rez1'] < df["Rez2"]),
    (df['H'] == tim) & (df['Rez1'] == df["Rez2"]),
    (df['A'] == tim) & (df['Rez1'] == df["Rez2"]),
    (df['H'] == tim) & (df['Rez1'] > df["Rez2"]),
    (df['A'] == tim) & (df['Rez1'] > df["Rez2"]),]
    
    vrijednosti = [-1, 1, 0, 0, 1, -1]
    df['Forma'] = np.select(uslovi, vrijednosti, default=0)
    df = df.drop(["Rez1","Rez2"],axis=1).tail(5)
    rezultat = df["Forma"].sum() / len(df["Forma"])
    return rezultat

## Provjera Rada te Funckije

In [7]:
klub('Real Madrid')

0.6

## Stvaranje Petlje za Automatski Izracun Forme Svih Klubova

In [8]:
#izvlacenje naziva svih klubova
timovi = pd.concat([df1["H"], df1["A"]]).unique()
timovi

array(['UD Almería', 'Sevilla FC', 'Real Sociedad', 'UD Las Palmas',
       'Athletic Club', 'RC Celta', 'Villarreal CF', 'Getafe CF',
       'Cádiz CF', 'Atlético de Madrid', 'RCD Mallorca', 'Valencia CF',
       'CA Osasuna', 'Girona FC', 'FC Barcelona', 'Real Betis',
       'Deportivo Alavés', 'Granada CF', 'Rayo Vallecano', 'Real Madrid',
       'Real Valladolid CF', 'RCD Espanyol de Barcelona', 'CD Leganés',
       'Elche CF', 'Levante UD', 'Real Oviedo'], dtype=object)

In [9]:
rezultati = {}

In [10]:
#petlja
for tim in timovi:
    rezultat = klub(tim)
    rezultati[tim] = rezultat

# Kreiranje DataFrame-a iz rezultata
df_forme = pd.DataFrame(list(rezultati.items()),columns = ["Klub","Rezultat"])
df_forme["Rezultat"] = df_forme["Rezultat"]

## Rezulat Forme za Sve

In [11]:
#Izracun
df_forme 

,Klub,Rezultat
0,UD Almería,0.0
1,Sevilla FC,-0.6
2,Real Sociedad,0.6
3,UD Las Palmas,-1.0
4,Athletic Club,-0.4
5,RC Celta,0.4
6,Villarreal CF,0.8
7,Getafe CF,0.0
8,Cádiz CF,0.0
9,Atlético de Madrid,1.0


In [12]:
#Ponovna provjera vrijednosti...
klub("Real Madrid")

0.6

In [13]:
df_forme.to_csv("forma_h.csv")

In [14]:
df_forme["Rezultat"] = df_forme["Rezultat"] * -1

In [15]:
df_forme


,Klub,Rezultat
0,UD Almería,-0.0
1,Sevilla FC,0.6
2,Real Sociedad,-0.6
3,UD Las Palmas,1.0
4,Athletic Club,0.4
5,RC Celta,-0.4
6,Villarreal CF,-0.8
7,Getafe CF,-0.0
8,Cádiz CF,-0.0
9,Atlético de Madrid,-1.0


In [16]:
df_forme.to_csv("forma_a.csv")